<a href="https://colab.research.google.com/github/Yassmina-Abdo/natural-language-processing/blob/main/tweets_classification_logisticReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets.
0 -> no hate speech
1 -> contains hate speech

**Total Estimated Time = 90 Mins**

### Import Libraries

In [1]:
!pip install contractions --q

     |████████████████████████████████| 106 kB 5.1 MB/s 
     |████████████████████████████████| 287 kB 43.0 MB/s 


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn import metrics
import contractions
import pandas as pd
import re

### Load Dataset

In [9]:
df = pd.read_csv('dataset.csv',index_col='id')

In [10]:
df.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


### EDA

- check NaNs

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31962 entries, 1 to 31962
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   31962 non-null  int64 
 1   tweet   31962 non-null  object
dtypes: int64(1), object(1)
memory usage: 749.1+ KB
None


- check duplicates

In [12]:
print(df.duplicated().sum())

2432


In [13]:
df=df.drop_duplicates()

In [14]:
print(df.duplicated().sum())

0


- show samples of data texts to find out required preprocessing steps

In [15]:
df.sample(5)

,label,tweet
id,,
4747,0,@user become happier with @user new app! chec...
22545,0,2 weeks to go before sdcc blues travel to leic...
5884,0,#ff ð @user @user @user @user @user @us...
18007,0,so my sis is back to work and my other sis is ...
16718,0,i can't deal with people who get sucked way in...


- check dataset balancing

In [16]:
labels=df.label.unique()
for label in labels:  
    print('Number of samples in class {} = {}'.format(label,len(df[df.label == label])))

Number of samples in class 0 = 27517
Number of samples in class 1 = 2013


- Cleaning and Preprocessing are:
    - 1 --> lower case
    - 2 --> remove contractions
    - 3 --> remove punc
    - 4 --> remove hash
    - 5 --> remove urls
    - 6 --> normalization

### Cleaning and Preprocessing

In [17]:
def preprocess(text): 
  text = text.lower() #1.lower
  text= contractions.fix(text)  #2.contractions 
  text = re.sub('@(\w+)',"", text )  #3. remove username
  text=  re.sub(r'[^\w\s]', '',  text)   #4. punc
  return text   

In [18]:
df['tweet'] = df['tweet'].apply(lambda x: preprocess(x))

In [19]:
df.sample(5)

,label,tweet
id,,
27905,0,motivationmonday your goals are impoant ð fit...
19312,1,only a racist would segregate people like y...
6523,0,thank you so much from all
6246,0,herbalremedies for ahritis that really stop pa...
5539,0,that stupid thane of cawdor how dare he betray...


## Modelling



*   BOW



In [20]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
X=vectorizer.fit_transform(df['tweet'])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, df['label'])

In [22]:
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')
lr.fit(X_train, y_train)
y_predict = lr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


 Evaluation

In [30]:
print("Accuracy %.3f" %metrics.accuracy_score(y_test, y_predict))
print(classification_report(y_test,y_predict ))

Accuracy 0.957
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6873
           1       0.83      0.47      0.60       510

    accuracy                           0.96      7383
   macro avg       0.90      0.73      0.79      7383
weighted avg       0.95      0.96      0.95      7383



### Enhancement

- Using different N-grams
- Using different text representation technique

In [31]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
X=vectorizer.fit_transform(df['tweet'])

X_train, X_test, y_train, y_test = train_test_split(X, df['label'])
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')
lr.fit(X_train, y_train)
y_predict = lr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [32]:
print("Accuracy %.3f" %metrics.accuracy_score(y_test, y_predict))
print(classification_report(y_test,y_predict ))

Accuracy 0.954
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6885
           1       0.83      0.40      0.54       498

    accuracy                           0.95      7383
   macro avg       0.89      0.70      0.76      7383
weighted avg       0.95      0.95      0.95      7383





*   TFIDF



In [33]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train, X_test, y_train, y_test = train_test_split(X, df['label'])
lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')
lr.fit(X_train, y_train)
y_predict = lr.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [34]:
print("Accuracy %.3f" %metrics.accuracy_score(y_test, y_predict))
print(classification_report(y_test,y_predict ))

Accuracy 0.954
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6878
           1       0.85      0.40      0.54       505

    accuracy                           0.95      7383
   macro avg       0.91      0.70      0.76      7383
weighted avg       0.95      0.95      0.95      7383



#### Done!